# Задание:
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

<h2>Сбор объектов и проверка что все они действительно имеют статус == "Строящиеся"</h2>

In [1]:
import requests
import pandas as pd

<h3>Ограничил запрос 5000 объектов т.к. при 10000 отваливался по таймауту</h3>

In [2]:
offset_ = 0
limit_ = 5000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

In [3]:
objects_list = objects_data.get('data').get('list')

In [4]:
df = pd.json_normalize(objects_list)

In [5]:
df.loc[df['siteStatus'] == 'Строящиеся']

,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13391,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,4293.67,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
1,13383,72,г Тюмень,4,20,68,2022-06-30,6289.98,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
2,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
3,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
4,13399,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,0.00,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11026,50,д Покровское,3,3,3,2022-12-31,713.40,1,0,...,Общество с ограниченной ответственностью,ООО,5.127020e+09,7707796333,"ЖК Футуро Парк (Futuro Park), Корпус №71",NaN,NaN,NaN,NaN,NaN
996,11023,50,д Покровское,2,3,7,2022-12-31,1526.20,1,0,...,Общество с ограниченной ответственностью,ООО,5.127020e+09,7707796333,"ЖК Футуро Парк (Futuro Park), Корпус №69",NaN,NaN,NaN,NaN,NaN
997,11025,50,д Покровское,3,3,5,2022-12-31,1188.90,1,0,...,Общество с ограниченной ответственностью,ООО,5.127020e+09,7707796333,"ЖК Футуро Парк (Futuro Park), Корпус №70",NaN,NaN,NaN,NaN,NaN
998,43491,76,"г Ярославль, пр-т Машиностроителей, д. 6,6, ли...",18,18,144,2023-09-30,8541.32,0,0,...,Общество с ограниченной ответственностью,ООО,4.549540e+08,7603066710,NaN,NaN,NaN,NaN,NaN,NaN


<h2>Сохраняем данные в разных форматах</h2>

In [6]:
df.to_excel('dmrf_excel.xlsx', index=False)

In [7]:
df.to_pickle('dmrf_pickle.pkl')

In [10]:
import sqlite3
connect = sqlite3.connect('dmrf_bd')

In [11]:
df.to_sql('obj', connect, if_exists='replace', index = False)
connect.commit()
connect.close()

# Задание *:
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [20]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/'
objID = []
for obj in objects_list:
    objID.append(obj['objId'])

<h3>Для 5 тысяч объектов фотографии будут грузиться очень долго. Вынес ограничитель в переменную count = 10. В данной конфигурации собирается список ссылок на фотографии только для 10 первых объектов.</h3>

In [21]:
def getImg(url, objID):
    url_list = []
    count = 10
    for id in objID:
        res = requests.get(url + str(id))
        for obj in res.json()['data']['photoRenderDTO']:
            url_list.append(obj['objRenderPhotoUrl'])
        count = count - 1
        if count == 0:
            break
    return url_list

In [22]:
url_list = getImg(url, objID)

In [23]:
import urllib.request
import os

<h3>В корне диска C:// создается директория img, в которую загружаются фотографии</h3>

In [27]:
imgName = 0
os.mkdir('C:/img')
for url in url_list:
    urllib.request.urlretrieve(url, 'C:/img/' + str(imgName) + '.jpg')
    imgName = imgName + 1